In [21]:
%reset -f
import ipyparallel
import os
rc = ipyparallel.Client(profile="mpi")
view = rc[:]
view.apply(os.chdir, os.getcwd())
view['stop'] = True
print view

view.block=True

<DirectView [0, 1, 2, 3,...]>


In [22]:
%%px --block
from mpi4py import MPI
mpi = MPI.COMM_WORLD
bcast = mpi.bcast
barrier = mpi.barrier
rank = mpi.rank


# Engines Imports
import sys
#!pwd   # Replace sys.path.append with this path
sys.path.append('/home/user/JorgeMijares/air-water-vv/2d/Interactive/Temp_Flume_GUI')

import proteus
## Required imports
from proteus.iproteus import * 
from proteus.mprans import SpatialTools as st
from petsc4py import PETSc
from threading import Thread
import tempFlume as plant
import tempFlume_so as plant_so
from proteus import default_n, default_s, default_so, Comm, Context
print "MPI rank: %i/%i" % (mpi.rank,mpi.size)

[stdout:0] MPI rank: 7/20
[stdout:1] MPI rank: 17/20
[stdout:2] MPI rank: 2/20
[stdout:3] MPI rank: 10/20
[stdout:4] MPI rank: 8/20
[stdout:5] MPI rank: 15/20
[stdout:6] MPI rank: 6/20
[stdout:7] MPI rank: 12/20
[stdout:8] MPI rank: 0/20
[stdout:9] MPI rank: 16/20
[stdout:10] MPI rank: 9/20
[stdout:11] MPI rank: 13/20
[stdout:12] MPI rank: 5/20
[stdout:13] MPI rank: 19/20
[stdout:14] MPI rank: 1/20
[stdout:15] MPI rank: 3/20
[stdout:16] MPI rank: 18/20
[stdout:17] MPI rank: 4/20
[stdout:18] MPI rank: 11/20
[stdout:19] MPI rank: 14/20


In [23]:
# Only Kernel Imports

import pandas as pd
from IPython.display import clear_output

# Kernel and Engines imports
#with view.sync_imports():  

    

## Simulation Parameters

In [24]:

#Parameters
dt=0.1            # Not sure if disabled, more testing needed, assign same value as dtOut too
cfl=0.75          # Not working yet, modify too in tempflume.py
dtOut=0.1          # Modify nDTout
T=60           # Total Simulation Time
Tank_he=0.03        # Domain he
Caisson_he=0.03     # Interpolation distance of caisson perimeter
enSphere=False        # Replace Caisson with sphere for testing of parameters
radius=0.1


#Select Case, if for loop deactivated
index  = 34             #Refer to DoE_TempFlume.xlsx to check case conditions
scale  = 21

## Monitor Functions

In [25]:
import numpy as np

def mpi_order(seq):
    """Return elements of a sequence ordered by MPI rank.

    The input sequence is assumed to be ordered by engine ID."""
    ranks = view['rank']
    rank_indices = np.argsort(ranks)
    return [seq[x] for x in rank_indices]


showTriangles = True
showAir = False


def plot_current_results():
    global t_0, plt0
    """Makes a blocking call to retrieve remote data and displays the solution mesh
    as a contour plot.

    Parameters
    ----------
    in_place : bool
        By default it calls clear_output so that new plots replace old ones.  Set
        to False to allow keeping of all previous outputs.
    """
    import numpy as np
    import matplotlib.tri as mtri
    import math
    global nn, x, y, vof, triangles, t, phi, u, v, cfl, p, dt
    view.apply_sync(load_simulation_globals)
    x = np.concatenate(mpi_order(view['x']))
    y = np.concatenate(mpi_order(view['y']))
    u = np.concatenate(mpi_order(view['u']))
    v = np.concatenate(mpi_order(view['v']))
    vof = np.concatenate(mpi_order(view['vof']))
    shifts = np.cumsum([0] + mpi_order(view['nn'])[:-1])
    flat_triangles = np.concatenate([tri + shift for tri, shift in zip(mpi_order(view['triangles']), shifts)])

    t = mpi_order(view['t'])[0]
    phi = np.concatenate(mpi_order(view['phi']))
    cfl = mpi_order(view['cfl'])[0]
    p = np.concatenate(mpi_order(view['p']))
    dt = mpi_order(view['dt'])[0]

    Vmax = np.amax(np.sqrt(u[:] ** 2 + v[:] ** 2))
    print "u_max={0:.3f}, v_max={1:.3f}, Vmax={2:.3f}, cfl={3:.3f}, dt={4:.5f}".format(np.amax(u[:]),
                                                                                       np.amax(v[:]),
                                                                                       Vmax,
                                                                                       np.asarray(cfl).max() * dt,
                                                                                       dt)                                                                                       

def load_simulation_globals():
    """Put some variables we need in engine namespace.

    These can then be retrieved by clients for inspection, visualization, etc.
    """
    global nn, x, y, vof, triangles, t, phi, u, v, cfl, p, dt
    model_vof = ns.modelList[1].levelModelList[-1]
    model_ls = ns.modelList[2].levelModelList[-1]
    # save solution and grid data for plotting purposes
    cfl = ns.modelList[0].levelModelList[-1].q[('cfl', 0)]
    dt = ns.systemStepController.dt_system
    x = ns.modelList[0].levelModelList[-1].mesh.nodeArray[:, 0]
    y = ns.modelList[0].levelModelList[-1].mesh.nodeArray[:, 1]
    triangles = ns.modelList[0].levelModelList[-1].mesh.elementNodesArray
    p = ns.modelList[0].levelModelList[-1].u[0].dof
    u = ns.modelList[0].levelModelList[-1].u[1].dof
    v = ns.modelList[0].levelModelList[-1].u[2].dof
    vof = ns.modelList[1].levelModelList[-1].u[0].dof_last
    phi = ns.modelList[2].levelModelList[-1].u[0].dof_last
    nn = len(x)
    # print "p={0}, u={1}, v={2}, triangles={3}, vof={4}, phi={5}".format(len(p),len(u),len(v),len(triangles),len(vof),len(phi))
    t = ns.systemStepController.t_system
    cfl *= dt


def simulation_alive():
    """Return True if the simulation thread is still running on any engine.
    """
    return any(view.apply_sync(lambda: simulation_thread.is_alive()))


def monitor_simulation(refresh=5.0):
    global t_0
    """Monitor the simulation progress and call plotting routine.

    Supress KeyboardInterrupt exception if interrupted, ensure that the last 
    figure is always displayed and provide basic timing and simulation status.

    Parameters
    ----------
    refresh : float
      Refresh interval between calls to retrieve and plot data.  The default
      is 5s, adjust depending on the desired refresh rate, but be aware that 
      very short intervals will start having a significant impact.

    """
    import datetime as dt, time
    if not simulation_alive():
        plot_current_results()
        print 'Simulation has already finished, no monitoring to do.'
        error = True
        return error
    t_0 = 0.
    t0 = dt.datetime.now()
    try:
        while simulation_alive():
            plot_current_results()
            error = False
            tmon = dt.datetime.now() - t0
            t_sim = view.apply_sync(lambda: ns.systemStepController.t_system)[0]
            print 'Monitored for: %s. at t=%12.5e' % (tmon, t_sim)
            time.sleep(refresh)  # so we don't hammer the server too fast
    except (KeyboardInterrupt):  # , error.TimeoutError):
        msg = 'Monitoring interrupted, simulation is ongoing!'
    else:
        t_sim = view.apply_sync(lambda: ns.systemStepController.t_system)[0]
        if not view.apply_sync(lambda: ns.systemStepController.converged()):
            msg = "\x1b[31mStep Failed at t=%12.5e \x1b[0m" % (t_sim)
            error = True
        else:
            msg = 'Simulation completed!'

    tmon = dt.datetime.now() - t0
    print msg
    print 'Monitored for: %s.' % tmon
    return error




## Simulation Setup

In [26]:

#Experimental Case

Scales = [21,25]        # Scale could be 21 or 25

#for s_ind, scale in enumerate(Scales):
if True:
        xl    = pd.ExcelFile("DoE_TempFlume.xlsx")
        df1   = xl.parse(0)
        df2   = xl.parse(1)
        df3   = xl.parse(2)
        df4   = xl.parse(3, header=0, index_col=0)
        df5   = xl.parse(4, header=0, index_col=0)
        

        clear_output()

        if scale == 21:
            df_DoE  = df4
            df_data = df2

        elif scale==25:
            df_DoE  = df5
            df_data = df3
        else:
            print 'Wrong Scale'
            () + 1
        #for index in range(df4.shape[0]):
            view['stop']=True
        if True:
            clear_output()
            test_name     = 'Results/S{0}T{1}'.format(scale,index)
            status        = df_DoE['Finished'].tolist()[index]
            #if status == 'Completed':
            #    continue
            
            # Read Index from DoE table
            iFlow   = df_DoE['Flow Speed'].tolist()[index]
            iDepth  = df_DoE['Depth'].tolist()[index]
            iOffset = df_DoE['Offset'].tolist()[index]
            
            # Read Tag parameters from table
            tagFlow   = df_data['Speed Tags'].tolist()[iFlow]
            tagDepth  = df_data['Depth Tags'].tolist()[iDepth]
            tagOffset = df_data['Offset Tags'].tolist()[iOffset]
            
            # Read parameters from table
            flowSpeed  = df_data['Flow Speed'].tolist()[iFlow]
            waterLevel = df_data['Depth'].tolist()[iDepth]
            offset     = df_data['Offset'].tolist()[iOffset]
            
            #Change Context
            
            def Reload_Context():
                    reload(Context)
                    reload(plant)
                    reload(plant_so)
                    
            view.apply_sync(Reload_Context)
            view['plant.opts.water_level'] = waterLevel
            view['plant.opts.tank_dim']    = (3.0,waterLevel+0.5)

            # plant.opts.wind_velocity=(flowSpeed,0.)
            view['plant.opts.inflow_velocity'] = flowSpeed
            # plant.opts.outflow_velocity=flowSpeed
            view['plant.opts.caisson_scale']   = float(scale)
            view['plant.opts.caisson_Yoffset'] = -offset
            
            view['plant.opts.dtOut']        = dtOut
            view['plant.opts.sphere']       = enSphere
            view['plant.opts.sphereRadius'] = radius
            view['plant.opts.dt_fixed']     = dt
            view['plant.opts.cfl']          = cfl
            view['plant.opts.T']            = T
            view['plant.opts.he']           = Tank_he
            view['plant.opts.he_caisson']   = Caisson_he

            print 'Scale={0}, Test={1}'.format(scale,index)
            print 'No Scaled values:'
            print 'Speed(ft/s)={0}, Depth(ft)={1}, Weigth(lb)={2}'.format(
                                                tagFlow,
                                                tagDepth,
                                                tagOffset)
            print 'Simulated Values:'
            print 'Speed={0}, Depth={1}, Offset={2}'.format(
                                                flowSpeed,
                                                waterLevel,
                                                -offset)
            def Setup_Simulation():
                global ct,ns, simulation_thread
                
                plant.Update_Model()
                Context.setFromModule(plant,mutable=True)
                
                plant_so.ct = Context.get()
                
                so = plant_so
                so.tnList = [0.0]+[i*plant.opts.dtOut for i in range(1,plant.nDTout+1)]        
                info = open("TimeList.txt","w")
                for time in so.tnList:
                    info.write(str(time)+"\n")
                info.close()
                plant_so.tnList=so.tnList
                pList=[]
                nList=[]
                so.sList=[]
                OptDB = PETSc.Options()
                for (p,n) in so.pnList:
                    so.sList.append(default_s)
                    pList.append(__import__(p))
                    reload(pList[-1])
                    nList.append(__import__(n))
                    reload(nList[-1])
                    pList[-1].name = p
                    nList[-1].multilevelLinearSolver = default_n.KSP_petsc4py
                    nList[-1].levelLinearSolver = default_n.KSP_petsc4py
                    OptDB.setValue(nList[-1].linear_solver_options_prefix+"ksp_type", "preonly")
                    OptDB.setValue(nList[-1].linear_solver_options_prefix+"pc_type", "lu")
                    OptDB.setValue(nList[-1].linear_solver_options_prefix+"pc_factor_mat_solver_package","superlu_dist")
                opts.save_dof = True
                opts.dataDir='.'
                opts.probDir='.'
                opts.logLevel=7
                opts.verbose=True
                opts.viewMesh=True


                ## Create numerical solution
                ns = NumericalSolution.NS_base(so, pList, nList, so.sList, opts)


                ## Start Simulation
                simulation_thread = Thread(target = lambda : ns.calculateSolution('run1'))
                simulation_thread.start()
                
            view.apply_sync(Setup_Simulation)
            print 'Simulation Started'
            
            
            ## Monitor Output
            error=monitor_simulation(refresh=5.0)      
            
            ## Export/Save Data
            import os   
            print test_name
            
            test_name='Results/S{0}T{1}'.format(scale,index)
            checkpath=os.path.exists('./'+test_name) 
            if not checkpath:
                    os.mkdir(test_name)
            with open(test_name+'/opts.txt', 'w+') as fp:
                opts = view.apply_sync(lambda : plant.opts._asdict().items())[0]
                fp.write('\n'.join('%s %s' % (key , val) for key, val in opts))

            !cp tempFlume_p.xmf ./{test_name}/tempFlume_p.xmf
            !cp tempFlume_p.h5  ./{test_name}/tempFlume_p.h5
            !cp forceHistory_p.txt ./{test_name}/forceHistory_p.txt
            !cp forceHistory_v.txt  ./{test_name}/forceHistory_v.txt
            !cp TimeList.txt ./{test_name}/TimeList.txt
            if not error:
                Exp_Status='Completed'
            else:
                Exp_Status='Crashed'

            print Exp_Status

            # Create a Pandas Excel writer using XlsxWriter as the engine.
            writer = pd.ExcelWriter('DoE_TempFlume.xlsx', engine='xlsxwriter')

            if scale==21:
                df4.loc[index,'Finished']=Exp_Status
                df4.loc[index,'Time']=T
            elif scale==25:
                df5.loc[index,'Finished']=Exp_Status
                df5.loc[index,'Time']=T
                print 'Scale Error'

            # Convert the dataframe to an XlsxWriter Excel object.
            df1.to_excel(writer, sheet_name='Parameters')
            df2.to_excel(writer, sheet_name='Parameters Scale 1-21')
            df3.to_excel(writer, sheet_name='Parameters Scale 1-25')
            df4.to_excel(writer, sheet_name='Tests Scale 1-21')
            df5.to_excel(writer, sheet_name='Tests Scale 1-25')
            # Close the Pandas Excel writer and output the Excel file.
            writer.save()

Scale=21, Test=34
No Scaled values:
Speed(ft/s)=18, Depth(ft)=15.0, Weigth(lb)=0.5 lbs
Simulated Values:
Speed=1.19723063299, Depth=0.217714285714, Offset=-0.006048
Simulation Started
u_max=2.071, v_max=0.931, Vmax=2.075, cfl=0.089, dt=0.00986
Monitored for: 0:00:31.477051. at t= 7.86939e-02
u_max=2.231, v_max=0.714, Vmax=2.232, cfl=0.058, dt=0.00578
Monitored for: 0:01:02.433895. at t= 1.58123e-01
u_max=2.232, v_max=0.813, Vmax=2.232, cfl=0.099, dt=0.00964
Monitored for: 0:01:39.366974. at t= 2.66929e-01
u_max=2.241, v_max=1.002, Vmax=2.241, cfl=0.151, dt=0.00947
Monitored for: 0:02:11.255035. at t= 3.43023e-01
u_max=2.327, v_max=1.179, Vmax=2.328, cfl=0.182, dt=0.00908
Monitored for: 0:02:45.970358. at t= 4.37366e-01
u_max=2.395, v_max=1.003, Vmax=2.396, cfl=0.173, dt=0.00778
Monitored for: 0:03:21.247033. at t= 5.27781e-01
u_max=2.252, v_max=0.879, Vmax=2.254, cfl=0.213, dt=0.00945
Monitored for: 0:03:59.122337. at t= 6.27632e-01
u_max=2.335, v_max=0.829, Vmax=2.336, cfl=0.204, dt=0

Monitored for: 0:43:18.406904. at t= 6.47579e+00
u_max=2.365, v_max=0.907, Vmax=2.365, cfl=0.359, dt=0.00841
Monitored for: 0:43:55.065997. at t= 6.56726e+00
u_max=2.440, v_max=0.894, Vmax=2.441, cfl=0.357, dt=0.00840
Monitored for: 0:44:30.458279. at t= 6.65873e+00
u_max=2.367, v_max=0.879, Vmax=2.367, cfl=0.349, dt=0.00837
Monitored for: 0:45:05.477424. at t= 6.75016e+00
u_max=2.411, v_max=1.071, Vmax=2.412, cfl=0.337, dt=0.00835
Monitored for: 0:45:43.239943. at t= 6.84175e+00
u_max=2.461, v_max=1.419, Vmax=2.462, cfl=0.333, dt=0.00835
Monitored for: 0:46:15.646870. at t= 6.92508e+00
u_max=2.411, v_max=1.612, Vmax=2.412, cfl=0.339, dt=0.00836
Monitored for: 0:46:52.726875. at t= 7.00837e+00
u_max=2.343, v_max=1.859, Vmax=2.344, cfl=0.344, dt=0.00838
Monitored for: 0:47:24.017243. at t= 7.09226e+00
u_max=2.839, v_max=1.986, Vmax=2.879, cfl=0.352, dt=0.00841
Monitored for: 0:47:59.818660. at t= 7.18427e+00
u_max=2.410, v_max=2.006, Vmax=2.411, cfl=0.310, dt=0.00726
Monitored for: 0:48

u_max=2.496, v_max=1.227, Vmax=2.497, cfl=0.350, dt=0.00804
Monitored for: 1:27:45.805520. at t= 1.28000e+01
u_max=2.433, v_max=1.103, Vmax=2.434, cfl=0.365, dt=0.00805
Monitored for: 1:28:20.808219. at t= 1.28806e+01
u_max=2.462, v_max=1.026, Vmax=2.462, cfl=0.263, dt=0.00565
Monitored for: 1:28:54.409182. at t= 1.29648e+01
u_max=2.535, v_max=0.991, Vmax=2.536, cfl=0.391, dt=0.00812
Monitored for: 1:29:29.861752. at t= 1.30487e+01
u_max=2.533, v_max=0.940, Vmax=2.534, cfl=0.401, dt=0.00812
Monitored for: 1:30:06.498788. at t= 1.31406e+01
u_max=2.523, v_max=0.890, Vmax=2.524, cfl=0.401, dt=0.00812
Monitored for: 1:30:43.812613. at t= 1.32243e+01
u_max=2.582, v_max=0.812, Vmax=2.583, cfl=0.397, dt=0.00811
Monitored for: 1:31:18.787940. at t= 1.33081e+01
u_max=2.466, v_max=0.933, Vmax=2.467, cfl=0.381, dt=0.00808
Monitored for: 1:31:54.078826. at t= 1.33944e+01
u_max=2.479, v_max=0.649, Vmax=2.480, cfl=0.377, dt=0.00805
Monitored for: 1:32:31.732657. at t= 1.34804e+01
u_max=2.419, v_max=

Monitored for: 2:11:47.579736. at t= 1.91079e+01
u_max=2.523, v_max=1.729, Vmax=2.524, cfl=0.580, dt=0.00794
Monitored for: 2:12:23.469748. at t= 1.91954e+01
u_max=2.494, v_max=1.681, Vmax=2.494, cfl=0.564, dt=0.00797
Monitored for: 2:12:58.919663. at t= 1.92798e+01
u_max=2.556, v_max=1.784, Vmax=2.556, cfl=0.284, dt=0.00421
Monitored for: 2:13:34.594440. at t= 1.93560e+01
u_max=2.539, v_max=1.719, Vmax=2.540, cfl=0.509, dt=0.00800
Monitored for: 2:14:09.455740. at t= 1.94400e+01
u_max=2.515, v_max=1.628, Vmax=2.515, cfl=0.464, dt=0.00801
Monitored for: 2:14:47.320309. at t= 1.95321e+01
u_max=2.496, v_max=1.675, Vmax=2.497, cfl=0.391, dt=0.00803
Monitored for: 2:15:21.488923. at t= 1.96161e+01
u_max=2.514, v_max=1.720, Vmax=2.515, cfl=0.343, dt=0.00808
Monitored for: 2:15:53.784218. at t= 1.96887e+01
u_max=2.491, v_max=1.750, Vmax=2.491, cfl=0.306, dt=0.00815
Monitored for: 2:16:28.751292. at t= 1.97655e+01
u_max=2.538, v_max=1.810, Vmax=2.539, cfl=0.290, dt=0.00825
Monitored for: 2:17

u_max=2.600, v_max=1.465, Vmax=2.601, cfl=0.305, dt=0.00806
Monitored for: 2:57:43.117004. at t= 2.53484e+01
u_max=2.540, v_max=1.305, Vmax=2.541, cfl=0.285, dt=0.00810
Monitored for: 2:58:15.067603. at t= 2.54242e+01
u_max=2.582, v_max=1.425, Vmax=2.583, cfl=0.265, dt=0.00801
Monitored for: 2:58:49.011376. at t= 2.55000e+01
u_max=2.496, v_max=1.430, Vmax=2.497, cfl=0.290, dt=0.00822
Monitored for: 2:59:25.883133. at t= 2.55820e+01
u_max=2.550, v_max=1.411, Vmax=2.550, cfl=0.177, dt=0.00494
Monitored for: 2:59:58.526843. at t= 2.56569e+01
u_max=2.566, v_max=1.449, Vmax=2.566, cfl=0.241, dt=0.00809
Monitored for: 3:00:35.682194. at t= 2.57322e+01
u_max=2.483, v_max=1.365, Vmax=2.483, cfl=0.237, dt=0.00803
Monitored for: 3:01:08.558080. at t= 2.58080e+01
u_max=2.532, v_max=1.289, Vmax=2.532, cfl=0.229, dt=0.00797
Monitored for: 3:01:42.194179. at t= 2.58795e+01
u_max=2.544, v_max=1.268, Vmax=2.545, cfl=0.214, dt=0.00790
Monitored for: 3:02:15.942740. at t= 2.59550e+01
u_max=2.488, v_max=

Monitored for: 3:41:21.385899. at t= 3.15000e+01
u_max=2.474, v_max=1.796, Vmax=2.480, cfl=0.354, dt=0.00816
Monitored for: 3:41:57.206364. at t= 3.15885e+01
u_max=2.575, v_max=1.485, Vmax=2.576, cfl=0.241, dt=0.00573
Monitored for: 3:42:33.451437. at t= 3.16741e+01
u_max=2.492, v_max=1.483, Vmax=2.493, cfl=0.315, dt=0.00825
Monitored for: 3:43:07.810260. at t= 3.17577e+01
u_max=2.571, v_max=1.434, Vmax=2.572, cfl=0.323, dt=0.00822
Monitored for: 3:43:41.183992. at t= 3.18410e+01
u_max=2.528, v_max=1.334, Vmax=2.529, cfl=0.357, dt=0.00818
Monitored for: 3:44:11.321715. at t= 3.19163e+01
u_max=2.608, v_max=1.251, Vmax=2.608, cfl=0.389, dt=0.00814
Monitored for: 3:44:44.753618. at t= 3.19947e+01
u_max=2.517, v_max=1.259, Vmax=2.518, cfl=0.424, dt=0.00812
Monitored for: 3:45:18.152587. at t= 3.20811e+01
u_max=2.456, v_max=1.284, Vmax=2.456, cfl=0.255, dt=0.00537
Monitored for: 3:45:50.644745. at t= 3.21648e+01
u_max=2.533, v_max=1.297, Vmax=2.533, cfl=0.333, dt=0.00811
Monitored for: 3:46

u_max=2.606, v_max=2.035, Vmax=2.634, cfl=0.395, dt=0.00742
Monitored for: 4:24:18.230398. at t= 3.78495e+01
u_max=2.468, v_max=2.001, Vmax=2.469, cfl=0.437, dt=0.00854
Monitored for: 4:24:49.958223. at t= 3.79342e+01
u_max=2.453, v_max=1.980, Vmax=2.453, cfl=0.448, dt=0.00854
Monitored for: 4:25:24.035201. at t= 3.80255e+01
u_max=2.505, v_max=1.974, Vmax=2.506, cfl=0.422, dt=0.00843
Monitored for: 4:25:58.243506. at t= 3.81084e+01
u_max=2.510, v_max=1.988, Vmax=2.511, cfl=0.387, dt=0.00844
Monitored for: 4:26:34.893485. at t= 3.82000e+01
u_max=2.621, v_max=1.832, Vmax=2.622, cfl=0.373, dt=0.00834
Monitored for: 4:27:08.513674. at t= 3.82906e+01
u_max=2.562, v_max=1.612, Vmax=2.563, cfl=0.220, dt=0.00470
Monitored for: 4:27:42.400530. at t= 3.83667e+01
u_max=2.501, v_max=1.600, Vmax=2.502, cfl=0.415, dt=0.00834
Monitored for: 4:28:16.752462. at t= 3.84496e+01
u_max=2.567, v_max=1.666, Vmax=2.567, cfl=0.393, dt=0.00817
Monitored for: 4:28:50.807374. at t= 3.85339e+01
u_max=2.422, v_max=

u_max=2.493, v_max=1.350, Vmax=2.494, cfl=0.378, dt=0.00772
Monitored for: 5:07:36.594973. at t= 4.42000e+01
u_max=2.477, v_max=1.342, Vmax=2.478, cfl=0.381, dt=0.00765
Monitored for: 5:08:10.725022. at t= 4.42841e+01
u_max=2.546, v_max=1.326, Vmax=2.547, cfl=0.189, dt=0.00414
Monitored for: 5:08:44.069328. at t= 4.43612e+01
u_max=2.509, v_max=1.338, Vmax=2.510, cfl=0.295, dt=0.00767
Monitored for: 5:09:21.185343. at t= 4.44461e+01
u_max=2.462, v_max=1.755, Vmax=2.463, cfl=0.296, dt=0.00769
Monitored for: 5:09:54.492910. at t= 4.45231e+01
u_max=2.426, v_max=1.650, Vmax=2.427, cfl=0.327, dt=0.00770
Monitored for: 5:10:31.246345. at t= 4.46077e+01
u_max=2.533, v_max=1.323, Vmax=2.534, cfl=0.323, dt=0.00773
Monitored for: 5:11:06.553519. at t= 4.46928e+01
u_max=2.495, v_max=1.192, Vmax=2.496, cfl=0.319, dt=0.00774
Monitored for: 5:11:39.826142. at t= 4.47773e+01
u_max=2.511, v_max=1.179, Vmax=2.511, cfl=0.315, dt=0.00772
Monitored for: 5:12:16.694974. at t= 4.48617e+01
u_max=2.542, v_max=

u_max=2.493, v_max=1.754, Vmax=2.494, cfl=0.373, dt=0.00809
Monitored for: 5:50:19.415100. at t= 5.04404e+01
u_max=2.448, v_max=1.533, Vmax=2.449, cfl=0.332, dt=0.00808
Monitored for: 5:50:49.996142. at t= 5.05161e+01
u_max=2.407, v_max=1.336, Vmax=2.407, cfl=0.285, dt=0.00807
Monitored for: 5:51:23.108396. at t= 5.05944e+01
u_max=2.447, v_max=1.319, Vmax=2.447, cfl=0.260, dt=0.00807
Monitored for: 5:51:59.356111. at t= 5.06888e+01
u_max=2.538, v_max=1.523, Vmax=2.539, cfl=0.178, dt=0.00561
Monitored for: 5:52:35.847771. at t= 5.07726e+01
u_max=2.523, v_max=1.458, Vmax=2.523, cfl=0.246, dt=0.00805
Monitored for: 5:53:10.987067. at t= 5.08563e+01
u_max=2.489, v_max=1.344, Vmax=2.490, cfl=0.240, dt=0.00803
Monitored for: 5:53:41.287604. at t= 5.09321e+01
u_max=2.394, v_max=1.228, Vmax=2.395, cfl=0.250, dt=0.00802
Monitored for: 5:54:17.876268. at t= 5.10160e+01
u_max=2.527, v_max=1.477, Vmax=2.527, cfl=0.278, dt=0.00801
Monitored for: 5:54:54.425061. at t= 5.11080e+01
u_max=2.536, v_max=

Monitored for: 6:33:12.058707. at t= 5.66950e+01
u_max=2.554, v_max=1.550, Vmax=2.554, cfl=0.398, dt=0.00824
Monitored for: 6:33:46.037816. at t= 5.67824e+01
u_max=2.587, v_max=1.706, Vmax=2.588, cfl=0.228, dt=0.00465
Monitored for: 6:34:21.506504. at t= 5.68666e+01
u_max=2.499, v_max=1.643, Vmax=2.499, cfl=0.368, dt=0.00834
Monitored for: 6:34:54.445459. at t= 5.69497e+01
u_max=2.496, v_max=1.547, Vmax=2.497, cfl=0.348, dt=0.00833
Monitored for: 6:35:26.447089. at t= 5.70250e+01
u_max=2.502, v_max=1.536, Vmax=2.503, cfl=0.361, dt=0.00832
Monitored for: 6:36:00.650736. at t= 5.71083e+01
u_max=2.453, v_max=1.558, Vmax=2.454, cfl=0.362, dt=0.00834
Monitored for: 6:36:32.428293. at t= 5.71830e+01
u_max=2.520, v_max=1.609, Vmax=2.520, cfl=0.181, dt=0.00438
Monitored for: 6:37:04.658932. at t= 5.72670e+01
u_max=2.476, v_max=1.644, Vmax=2.476, cfl=0.335, dt=0.00840
Monitored for: 6:37:40.582427. at t= 5.73590e+01
u_max=2.672, v_max=1.664, Vmax=2.672, cfl=0.342, dt=0.00841
Monitored for: 6:38

In [27]:
view['stop']=True